In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

In [ ]:
!pip install pytesseract pymupdf

In [ ]:
import os
import re
import zipfile
import requests
from bs4 import BeautifulSoup
# from PyPDF2 import PdfReader
# from pdf2image import convert_from_path
import pytesseract
from urllib.parse import urljoin, urlparse
import fitz
from PIL import Image
import io

In [ ]:
def download_file(url, folder):
    local_filename = os.path.join(folder, url.split('/')[-1])
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filename

In [ ]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with fitz.open(pdf_path) as pdf:
            for page_num in range(pdf.page_count):
                page = pdf[page_num]

                page_text = page.get_text()
                if page_text.strip():
                    text += page_text

                # images = page.get_images(full=True)
                # for img_index, img in enumerate(images):
                #     xref = img[0]
                #     base_image = pdf.extract_image(xref)
                #     image_bytes = base_image["image"]

                #     img_pil = Image.open(io.BytesIO(image_bytes))
                #     image_text = pytesseract.image_to_string(img_pil, lang='eng')

                #     if image_text.strip():
                #         text += image_text

    except Exception as e:
        print(f"Ошибка при извлечении текста из {pdf_path}: {e}")
    return text

In [ ]:
def get_page_folder_name(url):
    path = urlparse(url).path.strip("/")
    segments = path.split("/")
    return segments[-1] if segments else "root"

In [ ]:
def scrape_and_download(url, base_url, zipf, root_folder, downloaded_files, visited_links):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    main_content = soup.find('main') or soup

    page_folder = get_page_folder_name(url)

    for link in main_content.find_all('a', href=True):
        href = link['href']
        full_url = href if href.startswith('http') else urljoin(base_url, href)

        if full_url in visited_links:
            continue
        visited_links.add(full_url)

        if full_url.endswith('.pdf'):
            pdf_filename = os.path.basename(full_url)

            if pdf_filename not in downloaded_files:
                print(f"Скачиваем PDF: {full_url}")
                local_pdf_path = download_file(full_url, root_folder)
                downloaded_files.add(pdf_filename)

                text = extract_text_from_pdf(local_pdf_path)
                text_filename = os.path.basename(local_pdf_path).replace('.pdf', '.txt')

                zip_pdf_path = os.path.join(page_folder, pdf_filename)
                zip_text_path = os.path.join(page_folder, text_filename)

                with open(os.path.join(root_folder, text_filename), 'w', encoding='utf-8') as text_file:
                    text_file.write(text)

                zipf.write(local_pdf_path, zip_pdf_path)
                zipf.write(os.path.join(root_folder, text_filename), zip_text_path)

        elif base_url in full_url:
            print(f"Идем по ссылке: {full_url}")
            scrape_and_download(full_url, base_url, zipf, root_folder, downloaded_files, visited_links)

In [40]:
start_url = "https://pstu.ru/"
base_url = "/".join(start_url.split("/")[:-1])
root_folder = "downloads"

if not os.path.exists(root_folder):
    os.makedirs(root_folder)

zip_filename = "output.zip"

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    downloaded_files = set() # Создаем множество для отслеживания загруженных файлов.
    visited_links = set() # Создаем множество для отслеживания посещенных ссылок.
    scrape_and_download(start_url, base_url, zipf, root_folder, downloaded_files, visited_links)

print(f"ZIP архив создан: {zip_filename}")

Идем по ссылке: https://pstu.ru/basic/glossary/
Идем по ссылке: https://pstu.ru/sveden/
Идем по ссылке: https://pstu.ru/about/
Идем по ссылке: https://pstu.ru/sveden/common/
Идем по ссылке: https://pstu.ru/sveden/struct/
Идем по ссылке: https://pstu.ru/sveden/document/
Идем по ссылке: https://pstu.ru/sveden/education/
Скачиваем PDF: https://pstu.ru/files/file/Abitur/БУПы СОО/opop-soo.pdf
Идем по ссылке: https://pstu.ru/files/file/Abitur/БУПы СОО/uchebnyj-plan-soo.xlsx
Идем по ссылке: https://pstu.ru/files/file/Abitur/БУПы СОО/rabochie-programmy-disciplin-soo.docx
Скачиваем PDF: https://pstu.ru/files/file/Abitur/БУПы СОО/kalendarnyj-grafik.pdf
Скачиваем PDF: https://pstu.ru/files/file/Abitur/БУПы/2020/{6ea8912d-e61f-4482-af34-fa4a2444b5d7}.pdf
Идем по ссылке: https://pstu.ru/files/file/Abitur/БУПы/2020/{6ea8912d-e61f-4482-af34-fa4a2444b5d7}.pdf.sig
Скачиваем PDF: https://pstu.ru/files/file/Abitur/БУПы/2020/{49bad6ff-5e98-4b07-83c3-e90160c13d54}.pdf
Идем по ссылке: https://pstu.ru/files/

/usr/lib/python3.10/html/parser.py:342: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  self.handle_startendtag(tag, attrs)


Скачиваем PDF: http://pstu.ru/files/file/adm/doc/polozhenie_o_trudoustroystve_inostrannyh_grazhdan_v_pnipu_2015.pdf
Идем по ссылке: https://pstu.ru/files/image/en_version/campus.png
Идем по ссылке: https://pstu.ru/_res/gallery/en/1938pic.jpg
Идем по ссылке: https://pstu.ru/_res/gallery/en/1939pic.jpg
Идем по ссылке: https://pstu.ru/_res/gallery/en/1940pic.jpg
Идем по ссылке: https://pstu.ru/_res/gallery/en/1941pic.jpg
Идем по ссылке: https://pstu.ru/_res/gallery/en/1942pic.jpg
Идем по ссылке: https://pstu.ru/_res/gallery/en/1943pic.jpg
Идем по ссылке: https://pstu.ru/_res/gallery/en/1944pic.jpg
Идем по ссылке: https://pstu.ru/_res/gallery/en/1945pic.jpg
Идем по ссылке: https://pstu.ru/_res/gallery/en/1946pic.jpg
Идем по ссылке: https://pstu.ru/_res/gallery/en/1947pic.jpg
Идем по ссылке: https://pstu.ru/_res/gallery/en/1948pic.jpg
Идем по ссылке: https://pstu.ru/_res/gallery/en/1949pic.jpg
Идем по ссылке: https://pstu.ru/_res/gallery/en/1950pic.jpg
Идем по ссылке: https://pstu.ru/_res/g

ConnectionError: HTTPSConnectionPool(host='proceedings.socar.azuploads', port=443): Max retries exceeded with url: /pdf/47/8.Yef-55-63.pdf (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7c2a6d85aa70>: Failed to resolve 'proceedings.socar.azuploads' ([Errno -2] Name or service not known)"))